In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_es = pd.DataFrame()
for i in range(1, 6):
    df_temp = pd.read_csv(f'ES_part_{i}.csv')
    df_es = pd.concat([df_es, df_temp], ignore_index=True)

df_surprise = pd.read_csv('US_economic_releases_events.csv')

df_surprise.drop(columns=['S', 'Month', 'Surv(A)', 'Surv(H)', 'Surv(L)',], inplace=True)

df_surprise.drop(columns=['Flag', 'Country/Region', 'Day', 'C', 'Category','Subcategory', 'Std Dev', 'Period', 'Actual'])

# Dropping all rows for which surprise column has NaN or 0 value
df_surprise.dropna(subset=['Surprise'], inplace=True)
df_surprise = df_surprise[df_surprise['Surprise'] != 0]

df_surprise.replace("--", pd.NA, inplace=True)

# Redoing dropping all rows for which surprise column has NaN or 0 value
df_surprise.dropna(subset=['Surprise'], inplace=True)
df_surprise = df_surprise[df_surprise['Surprise'] != 0]

# Convert 'Surprise' column to float
df_surprise['Surprise'] = pd.to_numeric(df_surprise['Surprise'], errors='coerce')

# Again filtering out rows where 'Surprise' is 0 or NaN
df_surprise = df_surprise[df_surprise['Surprise'] != 0].dropna(subset=['Surprise'])

df_surprise.dropna(subset=['Time'], inplace=True)

# Wincorsizing to get results between 0.5% and 99.5% percentile for Surprise values
lower_bound = df_surprise['Surprise'].quantile(0.005)
upper_bound = df_surprise['Surprise'].quantile(0.995)

df_surprise = df_surprise[(df_surprise['Surprise'] >= lower_bound) & (df_surprise['Surprise'] <= upper_bound)]

# Step 1: Ensure columns are strings
df_surprise['Date'] = df_surprise['Date'].astype(str)
df_surprise['Time'] = df_surprise['Time'].astype(str)

# Step 2: Handle missing times (if any)
df_surprise['Time'] = df_surprise['Time'].fillna('00:00:00')

# Step 3: Combine Date and Time into DateTime
df_surprise['DateTime'] = pd.to_datetime(
    df_surprise['Date'].str[:10] + ' ' + df_surprise['Time'],
    format='%Y-%m-%d %H:%M:%S',
    errors='coerce'  # Converts invalid parsing to NaT instead of raising error
)

# First we drop Date and DateTime and change the column name for Unnamed: 0 to Date

df_surprise.drop(columns=['Date', 'DateTime'], inplace=True)
df_surprise.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)

# Now let's again create a DateTime column with the new Date column and check for number of NaN values
# Step 1: Ensure columns are strings
df_surprise['Date'] = df_surprise['Date'].astype(str)
df_surprise['Time'] = df_surprise['Time'].astype(str)

# Step 2: Handle missing times (if any)
df_surprise['Time'] = df_surprise['Time'].fillna('00:00:00')

# Step 3: Combine Date and Time into DateTime
df_surprise['DateTime'] = pd.to_datetime(
    df_surprise['Date'].str[:10] + ' ' + df_surprise['Time'],
    format='%Y-%m-%d %H:%M:%S',
    errors='coerce'  # Converts invalid parsing to NaT instead of raising error
)

# Step 1: Ensure columns are strings
df_es['Date'] = df_es['Date'].astype(str)
df_es['Time'] = df_es['Time'].astype(str)

# Step 2: Handle missing times (if any) and pad with seconds
df_es['Time'] = df_es['Time'].fillna('00:00')  # Fill missing times
df_es['Time'] = df_es['Time'] + ':00'  # Add seconds to make HH:MM:SS format

# Step 3: Combine Date and Time into DateTime with correct format
df_es['DateTime'] = pd.to_datetime(
    df_es['Date'] + ' ' + df_es['Time'],
    format='%m/%d/%Y %H:%M:%S',  # Matches MM/DD/YYYY date and HH:MM:SS time
    errors='coerce'
)

# Now we create the merged dataframe for our analysis - to allow us to match times of surprise with the price of the future at the time
df_combined = pd.merge(
    df_es,
    df_surprise,
    on='DateTime',
    how='outer',
    suffixes=('_es', '_surprise'),
    indicator=True  # this shows the source of each using suffix
)

# Some surprise announcements might have come before the starting point for the data on the futures, these would be meaningless for our analysis and should thus
# be removed by removing all rows with NaN values for Open

df_combined.dropna(subset=['Open'], inplace=True)


C:\Users\Jop Brouwer\AppData\Local\Temp\ipykernel_2976\2381584617.py:10: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_surprise = pd.read_csv('US_economic_releases_events.csv')
C:\Users\Jop Brouwer\AppData\Local\Temp\ipykernel_2976\2381584617.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_surprise.replace("--", pd.NA, inplace=True)


In [4]:
df_combined

,Date_es,Time_es,Open,High,Low,Close,Volume,DateTime,Date_surprise,Period,...,Subcategory,R,Day,Surv(M),# Ests.,Std Dev,Surprise,Country/Region,Flag,_merge
2,09/10/1997,00:01:00,0.00,0.00,0.00,0.00,0.0,1997-09-10 00:01:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,09/10/1997,00:02:00,0.00,0.00,0.00,0.00,0.0,1997-09-10 00:02:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,09/10/1997,00:03:00,0.00,0.00,0.00,0.00,0.0,1997-09-10 00:03:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
5,09/10/1997,00:04:00,0.00,0.00,0.00,0.00,0.0,1997-09-10 00:04:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
6,09/10/1997,00:05:00,0.00,0.00,0.00,0.00,0.0,1997-09-10 00:05:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9695761,12/19/2024,15:56:00,5941.75,5942.00,5941.25,5941.75,318.0,2024-12-19 15:56:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
9695762,12/19/2024,15:57:00,5941.75,5942.00,5941.25,5941.50,386.0,2024-12-19 15:57:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
9695763,12/19/2024,15:58:00,5941.50,5941.50,5940.75,5941.00,484.0,2024-12-19 15:58:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only
9695764,12/19/2024,15:59:00,5940.75,5941.25,5940.75,5941.00,6462.0,2024-12-19 15:59:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,left_only


ANN

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam # Keep Adam import
from tensorflow.keras.callbacks import EarlyStopping
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier # Older TF versions
# Use scikeras.wrappers for newer TF versions if using GridSearchCV
from sklearn.model_selection import train_test_split # We will split manually by time
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score

# --- FIX for Eager Execution Error ---
# Explicitly enable eager execution which might be needed in some environments/versions
# Or when certain operations trigger issues in graph mode during fit/evaluate
try:
    tf.config.run_functions_eagerly(True)
    print("TensorFlow Eager Execution enabled.")
except AttributeError:
    print("tf.config.run_functions_eagerly is not available (likely TF 1.x). Ensure TF 2.x is installed for default eager execution.")

# Print TensorFlow version
print(f"TensorFlow Version: {tf.__version__}")
# --------------------------------------


# Install ta if you don't have it: pip install ta
# Or use another library or manual calculations for technical indicators
try:
    import ta
except ImportError:
    print("Technical Analysis library 'ta' not found. Installing...")
    # Attempt to install 'ta' using pip within the environment if possible
    # Note: Direct pip install within the script might not work in all environments.
    # If this fails, please install it manually using: pip install ta
    import subprocess
    import sys
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "ta"])
        import ta
        print("Successfully installed and imported 'ta'.")
    except Exception as e:
        print(f"Failed to install 'ta': {e}")
        print("Please install 'ta' manually: pip install ta")
        # As a fallback, provide dummy functions or stop execution if 'ta' is critical
        # For this example, we'll stop if 'ta' cannot be loaded or installed.
        sys.exit("Required library 'ta' could not be loaded or installed.")
try:
    import matplotlib.pyplot as plt
    PLOTTING_ENABLED = True
except ImportError:
    print("\nMatplotlib not found. Cannot generate plots. Install with: pip install matplotlib")
    PLOTTING_ENABLED = False


# --- Configuration ---
# Define the lookback periods for indicators
RSI_PERIOD = 14
SMA_PERIOD = 50
ADX_PERIOD = 20
CORR_PERIOD = 24
MOM_PERIOD = 20
# Define the prediction/holding windows after a surprise
PREDICTION_WINDOWS = [10, 20, 40] # minutes - Updated to list

# Define ANN parameters (based on the paper/prompt)
N_NEURONS = 100
ACTIVATION_HIDDEN = 'relu'
ACTIVATION_OUTPUT = 'sigmoid'
# OPTIMIZER = Adam() # REMOVE optimizer instance creation from global scope
LOSS_FUNCTION = 'binary_crossentropy'
METRICS = ['accuracy']
EPOCHS = 20 # Max epochs
BATCH_SIZE = 32 # Standard batch size
EARLY_STOPPING_PATIENCE = 5 # Stop if validation loss doesn't improve for 5 epochs

# Define data split percentages
TRAIN_PCT = 0.30
VALIDATE_PCT = 0.20
# TEST_PCT = 0.50 # Remainder

# --- 1. Load Data ---
# Load the dataset provided by the user.
# The user specified using 'df_combined' which should be loaded from the CSV.
try:
    # Assuming the CSV file 'combined_datalast 10percdent.csv' is in the accessible path
    df_full = pd.read_csv('combined_datalast 10percdent.csv')
    print("Successfully loaded 'combined_datalast 10percdent.csv'.")
except FileNotFoundError:
    print("Error: 'combined_datalast 10percdent.csv' not found.")
    print("Please ensure the file is uploaded or in the correct directory.")
    # Exit or handle error appropriately
    # For demonstration, create a dummy dataframe if file not found
    # NOTE: Replace this with actual error handling if file is essential
    print("Creating dummy data for demonstration purposes.")
    dates = pd.to_datetime(pd.date_range(start='2023-01-01 09:00', periods=50000, freq='T'))
    price = 100 + np.random.randn(50000).cumsum() * 0.1
    volume = np.random.randint(100, 1000, 50000)
    df_full = pd.DataFrame({
        'DateTime': dates, # Ensure dummy data has 'DateTime' column
        'Open': price - np.random.rand(50000) * 0.1,
        'High': price + np.random.rand(50000) * 0.1,
        'Low': price - np.random.rand(50000) * 0.1,
        'Close': price,
        'Volume': volume,
        'Ticker': 'Dummy', # Assume a single instrument for simplicity
        'Event': np.nan,
        'Surprise': np.nan
    })
    # Add some dummy surprises
    surprise_indices = np.random.choice(df_full.index, size=100, replace=False)
    # Need to reset index if DateTime is not the index yet in dummy data
    if not isinstance(df_full.index, pd.DatetimeIndex):
         df_full = df_full.set_index('DateTime')
    df_full.loc[surprise_indices, 'Event'] = 'Dummy Event'
    df_full.loc[surprise_indices, 'Surprise'] = np.random.randn(100) * 0.5 # Standardized surprises


# --- 2. Preprocessing & Feature Engineering ---

# *** Explicitly use 'DateTime' column ***
datetime_col = 'DateTime'
print(f"Using specified column '{datetime_col}' for datetime index.")

# Check if the specified datetime column exists
if datetime_col not in df_full.columns:
    # If 'DateTime' is already the index (e.g. from dummy data creation)
    if df_full.index.name == datetime_col:
         print(f"Column '{datetime_col}' is already the index.")
         # No action needed if it's already the index
    else:
        raise ValueError(f"Specified datetime column '{datetime_col}' not found in the DataFrame. Available columns: {df_full.columns.tolist()}")
else:
    # Convert the specified column to datetime and set as index
    try:
        df_full[datetime_col] = pd.to_datetime(df_full[datetime_col])
        df_full = df_full.set_index(datetime_col)
        print(f"Column '{datetime_col}' successfully converted and set as index.")
    except Exception as e:
        raise ValueError(f"Could not parse the '{datetime_col}' column as datetime. Error: {e}")

df_full = df_full.sort_index() # Ensure chronological order

# Select necessary columns (adjust if names differ)
# Assuming standard OHLCV columns exist
ohlcv_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
surprise_cols = ['Ticker', 'Event', 'Surprise'] # Keep Ticker if multiple instruments exist

# Check for required columns
required_cols = ohlcv_cols + surprise_cols
missing_cols = [col for col in required_cols if col not in df_full.columns]
if missing_cols:
    raise ValueError(f"Missing required columns: {missing_cols}")

# Ensure numeric types for OHLCV and Surprise
for col in ohlcv_cols:
    df_full[col] = pd.to_numeric(df_full[col], errors='coerce')
df_full['Surprise'] = pd.to_numeric(df_full['Surprise'], errors='coerce')

# Drop rows with NaN in essential OHLCV columns
df_full.dropna(subset=ohlcv_cols, inplace=True)

print(f"Data shape after initial loading & cleaning: {df_full.shape}")
print(f"Time range: {df_full.index.min()} to {df_full.index.max()}")

# Calculate Technical Indicators using the 'ta' library
print("Calculating technical indicators...")
# RSI
df_full['RSI'] = ta.momentum.RSIIndicator(close=df_full['Close'], window=RSI_PERIOD).rsi()
# SMA
df_full['SMA'] = ta.trend.SMAIndicator(close=df_full['Close'], window=SMA_PERIOD).sma_indicator()
# ADX
adx_indicator = ta.trend.ADXIndicator(high=df_full['High'], low=df_full['Low'], close=df_full['Close'], window=ADX_PERIOD)
df_full['ADX'] = adx_indicator.adx()
# Momentum
df_full['Momentum'] = ta.momentum.ROCIndicator(close=df_full['Close'], window=MOM_PERIOD).roc()
# Correlation (e.g., with a benchmark or rolling self-correlation - let's use rolling return correlation)
# Calculate rolling return correlation with itself (measures serial correlation)
# Or potentially correlate with another asset if available. Using % change correlation.
df_full['Return'] = df_full['Close'].pct_change()
# Simple rolling std dev as a proxy if correlation is complex/unclear
df_full['Volatility'] = df_full['Return'].rolling(window=CORR_PERIOD).std()
# Using Volatility as 'Correlation' feature placeholder - replace if a specific correlation is needed
df_full['Correlation'] = df_full['Volatility']

# Previous Day's Price Differences
# Ensure we have daily data points or resample/forward-fill previous day's data
df_daily = df_full['Close'].resample('D').last()
df_daily_open = df_full['Open'].resample('D').first()
df_daily_high = df_full['High'].resample('D').max()
df_daily_low = df_full['Low'].resample('D').min()

# Calculate previous day's values - shift(1) gets the prior day
prev_close = df_daily.shift(1)
prev_open = df_daily_open.shift(1)
prev_high = df_daily_high.shift(1)
prev_low = df_daily_low.shift(1)

# Map previous day's values back to the minute data
df_full['Date'] = df_full.index.date
df_full['Prev_Close'] = df_full['Date'].map(prev_close)
df_full['Prev_Open'] = df_full['Date'].map(prev_open)
df_full['Prev_High'] = df_full['Date'].map(prev_high)
df_full['Prev_Low'] = df_full['Date'].map(prev_low)

# Calculate difference features
df_full['Prev_OC_Diff'] = df_full['Prev_Open'] - df_full['Prev_Close']
df_full['Prev_CH_Diff'] = df_full['Prev_Close'] - df_full['Prev_High']
df_full['Prev_CL_Diff'] = df_full['Prev_Close'] - df_full['Prev_Low']

# ---- Define Target Variables for Multiple Windows ----
print("Calculating target variables for multiple windows...")
for window in PREDICTION_WINDOWS:
    # Calculate the closing price 'window' minutes into the future
    df_full[f'Future_Close_{window}'] = df_full['Close'].shift(-window)
    # Calculate the price change percentage
    df_full[f'Future_Return_{window}'] = (df_full[f'Future_Close_{window}'] - df_full['Close']) / df_full['Close']
    # Create binary target: 1 if price goes up, 0 otherwise
    df_full[f'Target_{window}'] = (df_full[f'Future_Return_{window}'] > 0).astype(int)

# Filter for Surprise Events
# A surprise event is where the 'Surprise' column is not NaN
df_surprises = df_full.dropna(subset=['Surprise']).copy()

# --- Loop through each prediction window ---
results_summary = {} # Dictionary to store results for each window

for window in PREDICTION_WINDOWS:
    print(f"\n--- Processing for {window}-minute window ---")

    # Define Features (X) and Target (y) for the surprise events
    # Features: 9 indicators + Surprise value
    feature_cols = [
        'RSI', 'SMA', 'ADX', 'Volume', 'Correlation',
        'Prev_OC_Diff', 'Prev_CH_Diff', 'Prev_CL_Diff', 'Momentum',
        'Surprise' # Include the surprise value itself
    ]
    target_col = f'Target_{window}'
    future_return_col = f'Future_Return_{window}'

    # Select only the rows corresponding to surprises and the relevant columns
    # Include all feature cols, the specific target, and the specific future return
    relevant_cols = feature_cols + [target_col, future_return_col]
    df_model_data = df_surprises[relevant_cols].copy()

    # Drop rows with NaNs introduced by indicators or THIS SPECIFIC target calculation
    df_model_data.dropna(inplace=True)

    print(f"Shape of data for {window}-min window modeling: {df_model_data.shape}")

    if df_model_data.empty:
        print(f"Warning: No valid data points remaining for {window}-min window after NaN removal. Skipping this window.")
        results_summary[window] = {'status': 'Skipped - No data'}
        continue

    # Prepare X and y
    X = df_model_data[feature_cols]
    y = df_model_data[target_col]
    # Keep future returns for backtesting P&L calculation
    future_returns_for_backtest = df_model_data[future_return_col]
    timestamps_for_backtest = df_model_data.index # Keep timestamps for yearly analysis

    # --- 3. Data Splitting (Chronological) ---
    n_total = len(X)
    n_train = int(n_total * TRAIN_PCT)
    n_val = int(n_total * VALIDATE_PCT)
    n_test = n_total - n_train - n_val

    print(f"Total samples: {n_total}, Training: {n_train}, Validation: {n_val}, Test: {n_test}")

    if n_train == 0 or n_test == 0 or n_val == 0:
         print(f"Warning: Not enough data for the specified train/validation/test splits for {window}-min window. Skipping.")
         results_summary[window] = {'status': f'Skipped - Not enough data for split ({n_total} samples)'}
         continue

    X_train, X_val, X_test = X[:n_train], X[n_train:n_train+n_val], X[n_train+n_val:]
    y_train, y_val, y_test = y[:n_train], y[n_train:n_train+n_val], y[n_train+n_val:]
    timestamps_test = timestamps_for_backtest[n_train+n_val:]
    future_returns_test = future_returns_for_backtest[n_train+n_val:]

    print("Data split into training, validation, and test sets chronologically.")

    # --- 4. Scaling ---
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)

    print("Features scaled using MinMaxScaler.")

    # --- 5. ANN Model Definition ---
    def build_classifier(input_dim, window_suffix):
        model = Sequential(name=f"Macro_Surprise_ANN_{window_suffix}min")
        model.add(Input(shape=(input_dim,))) # Define input shape explicitly
        model.add(Dense(N_NEURONS, activation=ACTIVATION_HIDDEN))
        # Optional: Add dropout for regularization if needed
        # model.add(Dropout(0.2))
        model.add(Dense(1, activation=ACTIVATION_OUTPUT)) # Output layer for binary classification

        # *** FIX: Create a NEW optimizer instance HERE ***
        optimizer_instance = Adam()
        # *** --------------------------------------- ***

        model.compile(optimizer=optimizer_instance, # Use the new instance
                      loss=LOSS_FUNCTION,
                      metrics=METRICS)
        return model

    # Get input dimensions from scaled training data
    input_dim = X_train_scaled.shape[1]
    # Build a new model instance for this window
    tf.keras.backend.clear_session() # Clear previous models from memory
    model = build_classifier(input_dim, window)
    print(f"\nModel Summary for {window}-min window:")
    model.summary() # Print model architecture

    # --- 6. Model Training ---
    # Implement early stopping
    early_stopping = EarlyStopping(monitor='val_loss', # Monitor validation loss
                                 patience=EARLY_STOPPING_PATIENCE,
                                 restore_best_weights=True, # Restore weights from the best epoch
                                 verbose=1)

    print(f"Starting model training for {window}-min window...")
    # *** This is where the error occurred ***
    history = model.fit(X_train_scaled, y_train,
                        epochs=EPOCHS,
                        batch_size=BATCH_SIZE,
                        validation_data=(X_val_scaled, y_val),
                        callbacks=[early_stopping],
                        verbose=1) # Set verbose=1 or 2 for progress

    print(f"Model training finished for {window}-min window.")

    # --- 7. Evaluation ---
    print(f"\nEvaluating model on test data for {window}-min window...")
    loss, accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
    print(f"Test Loss: {loss:.4f}")
    print(f"Test Accuracy: {accuracy:.4f}")

    # Generate predictions for classification report
    y_pred_proba = model.predict(X_test_scaled)
    y_pred_binary = (y_pred_proba > 0.5).astype(int) # Threshold at 0.5

    print("\nClassification Report:")
    # Use try-except for report generation in case of issues with few samples
    try:
        report = classification_report(y_test, y_pred_binary, target_names=['Down (0)', 'Up (1)'], output_dict=True, zero_division=0)
        print(classification_report(y_test, y_pred_binary, target_names=['Down (0)', 'Up (1)'], zero_division=0))
    except Exception as e:
        print(f"Could not generate classification report: {e}")
        report = {"Error": str(e)}


    # --- 8. Backtesting Simulation ---
    print(f"\nSimulating trading strategy on test data for {window}-min window...")

    # Create a DataFrame for backtesting results
    backtest_results = pd.DataFrame({
        'Timestamp': timestamps_test,
        'Actual_Direction': y_test,
        'Predicted_Probability': y_pred_proba.flatten(),
        'Predicted_Direction': y_pred_binary.flatten(),
        'Future_Return': future_returns_test # Use the correct future return for this window
    })

    # Check if backtest_results is empty before proceeding
    if backtest_results.empty:
        print("No trades to simulate in backtest.")
        total_return = 0
        yearly_returns_dict = {}
    else:
        # Trading Logic:
        # Go Long if Predicted_Direction is 1 (Prob > 0.5)
        # Go Short if Predicted_Direction is 0 (Prob < 0.5)
        # P&L Calculation: Uses the future return corresponding to THIS window
        backtest_results['Strategy_Return'] = np.where(
            backtest_results['Predicted_Direction'] == 1, # Condition for Long
            backtest_results['Future_Return'],            # P&L if Long
            -backtest_results['Future_Return']            # P&L if Short
        )

        # Calculate Cumulative Returns
        backtest_results['Cumulative_Strategy_Return'] = (1 + backtest_results['Strategy_Return']).cumprod() - 1
        total_return = backtest_results['Cumulative_Strategy_Return'].iloc[-1]

        # Calculate Yearly Returns (if data spans multiple years)
        backtest_results['Year'] = backtest_results['Timestamp'].dt.year
        yearly_returns = backtest_results.groupby('Year')['Strategy_Return'].apply(lambda x: (1 + x).prod() - 1)
        yearly_returns_dict = {}
        if len(yearly_returns) > 0:
            print("\nApproximate Yearly Returns:")
            for year, ret in yearly_returns.items():
                print(f"  {year}: {ret:.2%}")
                yearly_returns_dict[year] = ret
        else:
            print("\nCould not calculate yearly returns (test data might not span a full year or more).")


    # --- 9. Results Storage ---
    print(f"\n--- Backtesting Results ({window}-min window) ---")
    print(f"Total Strategy Return on Test Set: {total_return:.2%}")

    # Store results
    results_summary[window] = {
        'status': 'Completed',
        'test_accuracy': accuracy,
        'classification_report': report,
        'total_return': total_return,
        'yearly_returns': yearly_returns_dict,
        'n_test_trades': len(backtest_results)
    }

    # Plotting cumulative returns (optional, requires matplotlib)
    if PLOTTING_ENABLED and not backtest_results.empty:
        plt.figure(figsize=(12, 6))
        plt.plot(backtest_results['Timestamp'], backtest_results['Cumulative_Strategy_Return'], label=f'Strategy Cumulative Return ({window} min)')
        plt.title(f'ANN Strategy Cumulative Returns ({window}-min window)')
        plt.xlabel('Date')
        plt.ylabel('Cumulative Return')
        plt.legend()
        plt.grid(True)
        # plt.show() # Display the plot
        # Instead of showing, save to file
        plot_filename = f'cumulative_returns_plot_{window}min.png'
        plt.savefig(plot_filename)
        print(f"\nCumulative returns plot saved to '{plot_filename}'.")
        plt.close() # Close the plot figure


# --- Final Summary ---
print("\n\n--- Overall Summary Across Windows ---")
for window, results in results_summary.items():
    print(f"\n--- Window: {window} minutes ---")
    if results['status'] == 'Completed':
        print(f"  Status: {results['status']}")
        print(f"  Test Accuracy: {results['test_accuracy']:.4f}")
        print(f"  Total Strategy Return: {results['total_return']:.2%}")
        print(f"  Number of Test Trades: {results['n_test_trades']}")
        print( "  Yearly Returns:")
        if results.get('yearly_returns'): # Use .get for safety
            for year, ret in results['yearly_returns'].items():
                print(f"    {year}: {ret:.2%}")
        else:
            print("    N/A")
        # Optionally print parts of the classification report again if needed
        # print(f"  Precision (Up): {results['classification_report']['Up (1)']['precision']:.2f}")
        # print(f"  Recall (Up): {results['classification_report']['Up (1)']['recall']:.2f}")
    else:
        print(f"  Status: {results['status']}")


print("\n--- Code and Strategy Explanation ---")
print("""
The code trains and evaluates separate Artificial Neural Network (ANN) models for predicting market direction over three different time horizons following a macroeconomic surprise: 10, 20, and 40 minutes.

**Error Fix 1**: Added `tf.config.run_functions_eagerly(True)` near the start to resolve a `NotImplementedError` related to NumPy conversion during model fitting.
**Error Fix 2**: Moved the `Adam()` optimizer instantiation *inside* the `build_classifier` function (within the processing loop). This ensures that each new model compiled gets a fresh optimizer instance, preventing the `ValueError: Unknown variable...` caused by optimizer state conflicts between models for different time windows.

1.  **Data Loading & Preparation**: Loads data, specifically looks for the 'DateTime' column to use as the time index, checks for its existence, parses it, sets it as the index, and sorts the data chronologically. Cleans data by ensuring numeric types and handling NaNs in OHLCV.
2.  **Feature Engineering**:
    * **Technical Indicators & Previous Day Features**: Calculated exactly as before (RSI, SMA, ADX, etc.). These features capture the market state *before* the surprise.
    * **Multiple Target Variables**: Instead of one target, the code now calculates three target variables (`Target_10`, `Target_20`, `Target_40`) and their corresponding future returns (`Future_Return_10`, etc.). Each target indicates whether the price went up (1) or down (0) over its specific time window (10, 20, or 40 minutes) after the surprise.
3.  **Processing Loop**: The core logic (data preparation for modeling, splitting, scaling, training, evaluation, backtesting) is placed inside a loop that iterates through the specified `PREDICTION_WINDOWS` (10, 20, 40).
4.  **Window-Specific Modeling**: Within each loop iteration:
    * The data (`df_model_data`) is prepared using the features and the *specific* target variable (`Target_window`) and future return (`Future_Return_window`) for the current window. Crucially, rows with NaNs resulting from the lookahead required for *that specific window* are dropped.
    * The data is split chronologically (Train/Val/Test) and scaled.
    * A *new instance* of the ANN model (with the same architecture: 100 neurons, ReLU/Sigmoid) is built. Crucially, a **new `Adam()` optimizer instance** is created and used when compiling this specific model. `tf.keras.backend.clear_session()` is also used to help ensure separation between models.
    * The model is evaluated on the test set for accuracy and other classification metrics.
5.  **Window-Specific Backtesting**:
    * The model trained for a specific window (e.g., 10 minutes) makes predictions on the test set.
    * The trading simulation (Long if P(Up) > 0.5, else Short) uses the predictions from the window-specific model.
    * The Profit & Loss (P&L) for each simulated trade is calculated using the actual price return over *that specific window* (e.g., `Future_Return_10` for the 10-minute model).
    * Cumulative and yearly returns are calculated for each window's strategy.
6.  **Results**: The script outputs the evaluation metrics (accuracy, classification report) and backtesting results (total and yearly returns) separately for each of the 10, 20, and 40-minute strategies. Plots for cumulative returns for each strategy are also saved if Matplotlib is installed. This allows comparing the effectiveness of the strategy across different holding periods.
""")

TensorFlow Eager Execution enabled.
TensorFlow Version: 2.19.0
Successfully loaded 'combined_datalast 10percdent.csv'.
Using specified column 'DateTime' for datetime index.
Column 'DateTime' successfully converted and set as index.
Data shape after initial loading & cleaning: (9695665, 30)
Time range: 1997-09-10 00:01:00 to 2024-12-19 16:00:00
Calculating technical indicators...
Calculating target variables for multiple windows...

--- Processing for 10-minute window ---
Shape of data for 10-min window modeling: (23037, 12)
Total samples: 23037, Training: 6911, Validation: 4607, Test: 11519
Data split into training, validation, and test sets chronologically.
Features scaled using MinMaxScaler.

Model Summary for 10-min window:


Model: "Macro_Surprise_ANN_10min"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 100)            │         1,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,201 (4.69 KB)

 Trainable params: 1,201 (4.69 KB)

...

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,404 (9.39 KB)

Starting model training for 20-min window...
Epoch 1/20


c:\Users\Jop Brouwer\Documents\GitHub\Data-Science-Seminar-Group-Project-5\.venv\Lib\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


ValueError: Unknown variable: <Variable path=Macro_Surprise_ANN_20min/dense/kernel, shape=(10, 100), dtype=float32, value=[[-0.15912457 -0.07778201 -0.11036867 -0.2084291   0.01118602  0.07709253
   0.14044595  0.12906677 -0.141143   -0.10201868 -0.06379631 -0.18756813
  -0.14059012 -0.21918686  0.03818512  0.16077071 -0.04443537  0.11455986
  -0.1637147  -0.0924841  -0.00245337 -0.07138312 -0.01098305  0.05244443
  -0.19269794 -0.02330846 -0.22716634  0.01932731  0.16841102 -0.05899431
   0.02472252  0.08458424 -0.1998723  -0.0263721  -0.22157125 -0.05893917
   0.05949757  0.22655344 -0.22641698  0.23168838 -0.09623949 -0.22026505
  -0.13087073 -0.03688776  0.21862108  0.1650708  -0.2318468   0.15136436
  -0.00342833 -0.15685278  0.00820594  0.1165579  -0.21798173  0.08096775
  -0.23066443  0.03088737  0.04200688 -0.01644334 -0.13526514  0.14054757
   0.13499653  0.01369642 -0.06750844 -0.0062526   0.18552127  0.17012522
   0.21003205  0.0147178  -0.20078377  0.07293499  0.11151463  0